In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import statsmodels


#creating a variable that holds the URL to the raw data stored on github
url = 'https://raw.githubusercontent.com/DMRocks/Olympic-Rowing-Data/main/Olympic%20Rowing%20Data?token=AEYUTCMT6AQ5GCWJZ7EBMVTBOYDKA'

#for more info on the read_csv command: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html
url = 'https://raw.githubusercontent.com/DMRocks/Olympic-Rowing-Data/main/Olympic%20Rowing%20Data'
data = pd.read_csv(url, index_col = 'Year')

data.head(10)

,Men's Single Gold,Men's Single Sliver,Men's Single Bronze,Men's Double Gold,Men's Double Sliver,Men's Double Bronze,Men's Quad Gold,Men's Quad Sliver,Men's Quad Bronze,Men's Four Gold,Men's Four Sliver,Men's Four Bronze,Men's Eight Gold,Men's Eight Sliver,Men's Eight Bronze
Year,,,,,,,,,,,,,,,
1920,455.00,456.00,468.00,429.00,439.00,441.00,NaN,NaN,NaN,NaN,NaN,NaN,363.60,365.00,396.00
1924,469.20,474.00,481.10,394.00,398.00,NaN,NaN,NaN,NaN,429.60,438.00,NaN,393.40,409.00,NaN
1928,431.00,441.80,450.80,401.40,411.00,409.80,NaN,NaN,NaN,396.00,397.00,392.60,363.20,366.60,364.80
1932,464.40,465.20,494.60,437.40,443.80,448.60,NaN,NaN,NaN,418.20,423.00,424.00,398.60,398.80,400.40
1936,502.50,506.80,508.00,441.80,446.20,456.20,NaN,NaN,NaN,422.80,427.50,431.60,385.40,386.00,386.40
1948,444.40,458.20,471.40,411.30,415.30,432.40,NaN,NaN,NaN,399.00,404.50,408.70,357.70,367.90,370.30
1952,493.80,495.50,499.40,452.20,458.30,464.70,NaN,NaN,NaN,436.00,439.90,443.30,386.90,391.20,393.10
1956,483.50,488.70,492.80,444.00,452.20,457.40,NaN,NaN,NaN,429.80,438.40,441.90,395.20,397.10,399.20
1960,434.96,440.21,441.26,408.50,410.49,411.59,NaN,NaN,NaN,386.26,389.78,390.62,357.18,362.52,365.84


# Plotly Graphs

In [14]:

events = data.columns

primer = 0


for i in range(5) :

    event_name = list(events[primer].split(" "))

    event_name = f"Men's {event_name[1]}"

    if (primer == 6):
        start_year = 1974
    else:
        start_year = 1916

    fig = px.scatter(data, x=data.index, y =events[primer:(primer+3)], 
    color_discrete_map={
            events[primer]: "gold",
            events[primer + 1]: "#d8d8d8",
            events[primer + 2]: "#CD7F32"
        },)


    fig.update_layout(
        title = f"Olympic {event_name} Final A Times (1920-2020)",
        yaxis_title = 'Olympic Year',
        xaxis_title = 'Final Time in Seconds',
        legend_title_text = 'Medal',
        yaxis = dict(
            tickmode = 'linear',
            #range= [],
            tick0 = 0,
            dtick = 10
        ),
        xaxis = dict(
            tickmode = 'linear',
            range=[start_year, 2024],
            tick0 = 0,
            dtick = 8
        )
    )

    primer = primer + 3

    fig.show()




# Dash 

In [134]:
import dash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input,Output
import pandas as pd
import plotly.express as px


url = 'https://raw.githubusercontent.com/DMRocks/Olympic-Rowing-Data/main/Olympic%20Rowing%20Data'
data = pd.read_csv(url, index_col = 'Year')

data.head(10)

external_stylesheets= ['https://codepen.io/chriddyp/pen/bWLwgP.css']

app = dash.Dash(__name__, external_stylesheets = external_stylesheets)
server = app.server


#App Layout
app.layout = html.Div([
    #Title
    html.H1("Olympic Rowing Times (Final A)", style={'text-align':'center'}),
    html.H5("Data Pulled from: Olympic Database, World Rowing and the New York Times"),
    html.Div("As you explore this dataset note not only how times get faster, but competition gets tighter."),
        #Dropdown Menu
    html.Div([
        dcc.Dropdown(
        id='event', clearable=False,
        value= f"Men's Single Gold", options=[
            {'label': c, 'value': c}
            for c in data.columns
        ], multi = True),
    ], style={'display': 'inline', 'width': '15%'}),
    #Main Graph
    html.Div([
        dcc.Graph(id='graph')
    ]),
    #Text Callbacks
    html.H3("How much faster have we become?", style={
        'text-align':'center'}),
    html.Div("To look at how much faster we have become, let us compare the top times from the eight event in 1920-1936 and the 2012-2020 Olympics (We take mean due to try to minimize the effect of wind). Here, as we can see a difference of 42 seconds."),
        
    html.Div([
        dcc.Graph(id='old_new')
    ]),

    html.Div("Here, take a look at times year by year and event by event."),
    #Add slider Sam
    dcc.Slider(
        id='bar_slider',
        min=1920,
        max=2024,
        step=4,
        value=1976,
        tooltip={"placement": "bottom", "always_visible": True},
        marks={
            1920: {'label': '1920', 'style': {'color': '#FFA500'}},
            1940: {'label': '1940', 'style': {'color': '#f50'}},
            1972: {'label': '1972', 'style': {'color': '#FFA500'}},
            2024: {'label': '2024'}
    }),
    html.Div([
        dcc.Graph(id='bar_chart')
    ]),    
])

# Graph Event
@app.callback(
    Output('graph', 'figure'),
    Input("event", "value")
)

def graph_callback(event):

    fig1 = px.scatter(data, x = data.index, y = event)
    
    fig1.update_layout(
    yaxis_title='Final Time (Seconds)',
    xaxis_title= 'Olympic Year',
    xaxis = dict(
        tickmode = 'linear',
        range=[1916, 2024],
        tick0 = 0,
        dtick = 8
        )
    )


    return fig1

#Fastest Time
@app.callback(
    Output(component_id='old_new', component_property='figure'),
    Input(component_id='event', component_property='value')
)
def update_output_div(event):

    data_time_comp_old = data.iloc[0:4, 12]

    data_time_comp_new = data.iloc[20:22, 12]

    old_mean = data_time_comp_old.mean()

    new_mean = data_time_comp_new.mean()

    mean_data = {'mean': [old_mean, new_mean]}

    mean_data = pd.DataFrame(mean_data, index=['1920-1936','2012-2020'])

    mean_bar_fig = px.bar(mean_data)


    mean_bar_fig.update_layout(
    title_text='Mean of Olympic Eight Gold Times (1920-1936, 2012-2020)',
    yaxis_title='Final Time (Seconds)',
    xaxis_title= 'Mean of Olympic Years',
    showlegend=False,
    yaxis = dict(
        tickmode = 'linear',
        range=[310, 390],
        tick0 = 0,
        dtick = 25
        )
)

    return mean_bar_fig

#Bar Chart
@app.callback(
    Output(component_id='bar_chart', component_property='figure'),
    Input("bar_slider", "value")
)
def run_bar_chart(slider):

    data_selection = data.loc[slider]

    bar_fig = px.bar(data_selection, barmode = 'group')


    bar_fig.update_layout(
    yaxis_title='Final Time (Seconds)',
    xaxis_title= 'Olympic Year',
    showlegend=False,
    yaxis = dict(
        tickmode = 'linear',
        range=[310, 510],
        tick0 = 0,
        dtick = 25
        )
    )


    return bar_fig






# Run app
if __name__ == '__main__':
    app.run_server()

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

Dash is run

 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [26/Oct/2021 18:51:51] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "GET /_favicon.ico?v=2.0.0 HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "GET /_dash-component-suites/dash/dcc/async-slider.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [26/Oct/2021 18:51:51] "POST /_dash-upd

# Testing and Time Difference 

In [132]:
import plotly.express as px


url = 'https://raw.githubusercontent.com/DMRocks/Olympic-Rowing-Data/main/Olympic%20Rowing%20Data'
data = pd.read_csv(url, index_col = 'Year')



data_time_comp_old = data.iloc[0:4, 12]

data_time_comp_new = data.iloc[20:22, 12]

old_mean = data_time_comp_old.mean()

new_mean = data_time_comp_new.mean()


print(old_mean - new_mean)

42.38000000000005


In [91]:

import plotly.express as px


url = 'https://raw.githubusercontent.com/DMRocks/Olympic-Rowing-Data/main/Olympic%20Rowing%20Data'
data = pd.read_csv(url, index_col = 'Year')

slider = 1976
event = "Men's Single Gold"

year_int = (slider - 1920) / 4

year_int = int(year_int)

data_selection = data.loc[slider]


print(data_selection)

bar_fig = px.bar(data_selection, barmode = 'group', color_discrete_sequence=["red", "blue"])


bar_fig.update_layout(
yaxis_title='Final Time (Seconds)',
xaxis_title= 'Olympic Year',
showlegend=False,
yaxis = dict(
    tickmode = 'linear',
    range=[310, 510],
    tick0 = 0,
    dtick = 25
    )
)

Men's Single Gold      449.03
Men's Single Sliver    452.67
Men's Single Bronze    458.03
Men's Double Gold      433.20
Men's Double Sliver    435.26
Men's Double Bronze    437.45
Men's Quad Gold        379.65
Men's Quad Sliver      380.89
Men's Quad Bronze      382.77
Men's Four Gold        397.42
Men's Four Sliver      401.22
Men's Four Bronze      403.52
Men's Eight Gold       358.29
Men's Eight Sliver     361.82
Men's Eight Bronze     364.51
Name: 1976, dtype: float64
